In [1101]:
!pip install cohens_d

In [1102]:
import pandas as pd
import numpy as np
import ast
import json
from collections import Counter, defaultdict

In [1103]:
#df_arguments_sentiment = pd.read_csv("sentiment_overview_3.5-turbo_v3.csv")
df_paragraphs_arguments = pd.read_csv("report_arguments_3.5_v3.csv")
df_reports = pd.read_csv("../argument_extraction/production/preprocessed_reports.csv")

In [1104]:
#df_arguments_sentiment = df_arguments_sentiment.drop(columns="Unnamed: 0")
df_paragraphs_arguments = df_paragraphs_arguments.drop(columns=["Unnamed: 0","Unnamed: 0.1","paragraph"])
df_reports = df_reports.drop(columns=["Unnamed: 0","paragraphs"])

In [1105]:
df_paragraphs_arguments

,filename,document_id,paragraph_id,provided_arguments
0,20140114_Needham_NVDA_NVDA-_Compelling_Technol...,512,1,"{'Technology Trends': 'positive', 'Market Expa..."
1,20140114_Needham_NVDA_NVDA-_Compelling_Technol...,512,2,"{'Technology Trends': 'positive', 'Partnership..."
2,20140114_Needham_NVDA_NVDA-_Compelling_Technol...,512,3,"{'Innovation': 'positive', 'Technology Trends'..."
3,20140114_Needham_NVDA_NVDA-_Compelling_Technol...,512,4,"{'Technology Trends': 'positive', 'Market Shar..."
4,20140114_Needham_NVDA_NVDA-_Compelling_Technol...,512,5,"{'Growth': 'negative', 'Technology Trends': 'n..."
...,...,...,...,...
12567,20221230_JP_Morgan_AAPL_Apple-_Public_CAICT_Ch...,203,1,"{'Market Share': 'positive', 'Revenue': 'negat..."
12568,20221230_JP_Morgan_AAPL_Apple-_Public_CAICT_Ch...,203,2,"{'Market Share': 'positive', 'Growth': 'positi..."
12569,20221230_JP_Morgan_AAPL_Apple-_Public_CAICT_Ch...,203,3,{'Market Share': 'positive'}
12570,20221230_JP_Morgan_AAPL_Apple-_Public_CAICT_Ch...,203,4,{'Growth': 'positive'}


<div style="background-color: #181818; padding: 5px; border-radius: 20px; text-align: center;">
    <h2 style="font-size: 1.7em;color:#FBFAFA">Argument Aggreation</h2>
</div>

In [1106]:
# Turn string dictionary back to actual dictionary
df_paragraphs_arguments["provided_arguments"] = df_paragraphs_arguments["provided_arguments"].apply(ast.literal_eval)

In [1107]:
#1. Step -> create additional columns that contain the arguments/sentiments as a list, this way the argument can be multiple times per document (not possibel in dictionary)
df_paragraphs_arguments['argument_list'] = df_paragraphs_arguments['provided_arguments'].apply(lambda x: list(x.keys()))
df_paragraphs_arguments['sentiment_list'] = df_paragraphs_arguments['provided_arguments'].apply(lambda x: list(x.values()))

In [1108]:
# Calculate unique amount of arguments from paragraphs
unique_arguments = set(value for sublist in df_paragraphs_arguments["argument_list"] for value in sublist)

print(f"Initial amounf of unique arguments {len(unique_arguments)}")

Initial amounf of unique arguments 385


In [1109]:
## Mapping before ratios are getting created
mapping = {
    "Growth":["Sales Growth","Earnings Growth"],
    "Price/Earnings Ratio":[],
    "Earnings per Share":["EPS"],
    "Cash Flow":["Cash Position","Cash Management","Liquidity","Working Capital","Free Cash Flow"],
    "Revenue":["Revenue Growth","Sales","Operating Income","Deferred Revenue","Revenue Recognition"],
    "Return On Equity":["Return On Invested Capital","Return On Tangible Common Equity","Return On Assets","Return On Capital Employed"],
    "Margins":["EBITDA","Gross Margin","Net Income","EBIT Margin","Gross Profits","Net Interest Margin","Margins Expansion","Gross Margin Return on Inventory","EBITDA Margin","Operating Margin"],
    "Cost Management":["Expenses","Operating Expenses","SG&A","Operating Expense","Spending","Cost Reduction"],
    "Dividend Policy":[],
    "Investments":["Capital Expenditure","Capex Allocation"],
    "Balance Sheet":["Capital Allocation","Debt Reduction","Capital Adequacy","Asset Sale","Equities","Asset Management","Loan Growth"],
    "Long-term Growth":[],
    "Mergers and Acquisition":["Acquisition","Acquisitions"],
    "Refranchising":["Franchise Management","Franchising"],
    "Sustainability":["Environmental Impact","Emissions Reduction","Energy Transition"],
    "Employees":["Intellectual Property","Talent Management","Labor Challenges","Employee Satisfaction"],
    "Research and Development":["Product Development","Drug Development","Product Launch"],
    "Marketing":["Advertising","Brand Image","Promotions","Sales & Marketing"],
    "Shares Repurchase":["Share Buyback","Stock Repurchase","Stock Buyback"],
    "Processes":["Productivity","Efficiency"],
    "Leadership":["Management"],
    "Innovation":["Product Innovation"],
    "Product Characteristics":[],
    "Pricing Strategy":["Price Impact"],
    "Production":[],
    "Technology Trends":["Digital Transformation","Digital Initiatives"],
    "Market Share":[],
    "Market Conditions":[],
    "Market Expansion":["International Expansion"],
    "Competition":["Competitive Position"],
    "Global Presence":["Global Expansion"],
    "Industry Outlook":["Market Trends"],
    "Regulations":["Regulatory Issues","Legal Issues","Privacy","Political Environment","Regulatory Approval"],
    "Partnerships and Collaborations":["Partnerships","Collaboration","Partnerships and Collaboration","Collaborations"],
    "Supply Chain":[],
    "Economic Conditions":["Interest Rates","Net Interest Income","Inflation", "Global Economic Conditions"],
    "Demand":["User Growth","Market Growth"],
    "Customer":["Customer Experience","Customer","Customer Acquisition","Customer Service","Customer Retention", "Customer Engagement","User Engagement","Customer","Customer Protection","Customers"],
    "Currency":["Currency Exchange","Currency Impact","Foreign Exchange"],
    "Taxes":["Tax Rate","Tax Policy Impact","Tax Liability","Tax Impact","Taxes"],
    "Inventory Management":["Inventory"],
    "Earnings Forecast":["Earnings Guidance"],
    "Financial Performance":["Financial Strength","Earnings Quality"]
}


reverse_mapping = {v: k for k, values in mapping.items() for v in values}

# Function to map arguments to their broader categories
def map_arguments(argument_list, reverse_mapping):
    return [reverse_mapping.get(arg, arg) for arg in argument_list]

# Apply the mapping function to the `argument_list` column
df_paragraphs_arguments["argument_list"] = df_paragraphs_arguments["argument_list"].apply(
    lambda x: map_arguments(x, reverse_mapping) if isinstance(x, list) else x
)


In [1110]:
df_paragraphs_arguments

,filename,document_id,paragraph_id,provided_arguments,argument_list,sentiment_list
0,20140114_Needham_NVDA_NVDA-_Compelling_Technol...,512,1,"{'Technology Trends': 'positive', 'Market Expa...","[Technology Trends, Market Expansion]","[positive, negative]"
1,20140114_Needham_NVDA_NVDA-_Compelling_Technol...,512,2,"{'Technology Trends': 'positive', 'Partnership...","[Technology Trends, Partnerships and Collabora...","[positive, positive]"
2,20140114_Needham_NVDA_NVDA-_Compelling_Technol...,512,3,"{'Innovation': 'positive', 'Technology Trends'...","[Innovation, Technology Trends, Competition]","[positive, positive, positive]"
3,20140114_Needham_NVDA_NVDA-_Compelling_Technol...,512,4,"{'Technology Trends': 'positive', 'Market Shar...","[Technology Trends, Market Share, Competition]","[positive, positive, negative]"
4,20140114_Needham_NVDA_NVDA-_Compelling_Technol...,512,5,"{'Growth': 'negative', 'Technology Trends': 'n...","[Growth, Technology Trends, Competition]","[negative, negative, negative]"
...,...,...,...,...,...,...
12567,20221230_JP_Morgan_AAPL_Apple-_Public_CAICT_Ch...,203,1,"{'Market Share': 'positive', 'Revenue': 'negat...","[Market Share, Revenue]","[positive, negative]"
12568,20221230_JP_Morgan_AAPL_Apple-_Public_CAICT_Ch...,203,2,"{'Market Share': 'positive', 'Growth': 'positi...","[Market Share, Growth]","[positive, positive]"
12569,20221230_JP_Morgan_AAPL_Apple-_Public_CAICT_Ch...,203,3,{'Market Share': 'positive'},[Market Share],[positive]
12570,20221230_JP_Morgan_AAPL_Apple-_Public_CAICT_Ch...,203,4,{'Growth': 'positive'},[Growth],[positive]


In [1111]:
from collections import Counter

# Function to count sentiments using `argument_list` and `sentiment_list`
def count_sentiments(argument_list, sentiment_list):
    counts = Counter(sentiment_list)  # Count occurrences of each sentiment
    return {
        'positive_arguments': counts.get('positive', 0),
        'neutral_arguments': counts.get('neutral', 0),
        'negative_arguments': counts.get('negative', 0)
    }

df_counts_paragraph = df_paragraphs_arguments.apply(
    lambda row: count_sentiments(row['argument_list'], row['sentiment_list']), axis=1
)

# Expand the dictionary into separate columns and concatenate with the original DataFrame
df_counts_paragraph = pd.DataFrame(df_counts_paragraph.tolist())
df_paragraphs_arguments = pd.concat([df_paragraphs_arguments, df_counts_paragraph], axis=1)

In [1112]:
df_paragraphs_arguments

,filename,document_id,paragraph_id,provided_arguments,argument_list,sentiment_list,positive_arguments,neutral_arguments,negative_arguments
0,20140114_Needham_NVDA_NVDA-_Compelling_Technol...,512,1,"{'Technology Trends': 'positive', 'Market Expa...","[Technology Trends, Market Expansion]","[positive, negative]",1,0,1
1,20140114_Needham_NVDA_NVDA-_Compelling_Technol...,512,2,"{'Technology Trends': 'positive', 'Partnership...","[Technology Trends, Partnerships and Collabora...","[positive, positive]",2,0,0
2,20140114_Needham_NVDA_NVDA-_Compelling_Technol...,512,3,"{'Innovation': 'positive', 'Technology Trends'...","[Innovation, Technology Trends, Competition]","[positive, positive, positive]",3,0,0
3,20140114_Needham_NVDA_NVDA-_Compelling_Technol...,512,4,"{'Technology Trends': 'positive', 'Market Shar...","[Technology Trends, Market Share, Competition]","[positive, positive, negative]",2,0,1
4,20140114_Needham_NVDA_NVDA-_Compelling_Technol...,512,5,"{'Growth': 'negative', 'Technology Trends': 'n...","[Growth, Technology Trends, Competition]","[negative, negative, negative]",0,0,3
...,...,...,...,...,...,...,...,...,...
12567,20221230_JP_Morgan_AAPL_Apple-_Public_CAICT_Ch...,203,1,"{'Market Share': 'positive', 'Revenue': 'negat...","[Market Share, Revenue]","[positive, negative]",1,0,1
12568,20221230_JP_Morgan_AAPL_Apple-_Public_CAICT_Ch...,203,2,"{'Market Share': 'positive', 'Growth': 'positi...","[Market Share, Growth]","[positive, positive]",2,0,0
12569,20221230_JP_Morgan_AAPL_Apple-_Public_CAICT_Ch...,203,3,{'Market Share': 'positive'},[Market Share],[positive],1,0,0
12570,20221230_JP_Morgan_AAPL_Apple-_Public_CAICT_Ch...,203,4,{'Growth': 'positive'},[Growth],[positive],1,0,0


In [1113]:
df_counts_reports = df_paragraphs_arguments.groupby('filename',as_index=False)[['positive_arguments','negative_arguments','neutral_arguments']].sum()
df_reports = df_reports.merge(df_counts_reports,on="filename",how='inner',)
df_reports["tp_reached"] = df_reports[["tp reached after 3 months","tp reached after 6 months","tp reached after 9 months","tp reached after 12 months"]].any(axis=1)
df_reports["total_arguments"] = df_reports[["positive_arguments","negative_arguments","neutral_arguments"]].sum(axis=1)
df_reports

,document_id,filename,date,provider,ticker,company_name,industry,target_price,rating,start price,...,adjusted_target_price,tp reached after 3 months,tp reached after 6 months,tp reached after 9 months,tp reached after 12 months,positive_arguments,negative_arguments,neutral_arguments,tp_reached,total_arguments
0,1,20161018_Needham_META_Facebook-_3Q16_Preview_R...,2016-10-11,Needham,META,Meta Platforms Inc.,Communication Services,150.0,buy,128.88,...,150.00,False,False,True,True,15,3,2,True,20
1,2,20200807_Wells_Fargo_META_FB-_2.5B_Person_Plat...,2020-07-31,Wells Fargo,META,Meta Platforms Inc.,Communication Services,300.0,overweight,253.67,...,300.00,True,False,True,True,19,9,1,True,29
2,3,20200918_Barclays_GM_General_Motors-_Time_to_t...,2020-09-14,Barclays,GM,General Motors Co.,Automobile,39.0,overweight,31.18,...,39.00,True,True,True,True,21,0,1,True,22
3,4,20200722_Barclays_BKNG_Booking_Holdings_Inc.-_...,2020-07-16,Barclays,BKNG,Booking Holdings Inc,Travel,1950.0,overweight,1746.79,...,1950.00,False,True,True,True,7,5,2,True,14
4,5,20211104_Deutsche_Bank_X_US_Steel-_3Q21_EBITDA...,2021-10-28,Deutsche Bank,X,United States Steel,Materials,50.0,buy,23.38,...,50.00,False,False,False,False,15,2,1,False,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
911,913,20150522_Gilford_Securities_Inc_FL_Report_rece...,2015-05-22,Gilford Securities Inc,FL,Foot Locker,Clothing,NaN,buy,63.46,...,NaN,False,False,False,False,14,4,0,False,18
912,914,20200526_Wells_Fargo_HD_HD-_Q2_Rollover_Falls_...,2020-05-19,Wells Fargo,HD,Home Depot Inc.,Consumer Discretionary,270.0,overweight,238.10,...,270.00,True,True,True,True,15,3,1,True,19
913,915,20220125_Deutsche_Bank_WFC_Wells_Fargo-_More_C...,2022-01-17,Deutsche Bank,WFC,Wells Fargo & Co.,Financials,55.0,buy,58.06,...,55.00,True,True,True,True,16,10,5,True,31
914,916,20200212_Barclays_GM_General_Motors-_Waiting_p...,2020-02-06,Barclays,GM,General Motors Co.,Automobile,44.0,overweight,34.33,...,44.00,False,False,False,True,38,12,9,True,59


In [1114]:
df_reports["total_arguments"].sum()

24874

In [1115]:
# merge dictionaries
def merge_lists(x):
    return [item for sublist in x.dropna() for item in sublist]

df_reports_arguments = df_paragraphs_arguments.groupby(['filename', 'document_id'], as_index=False).agg({
    'argument_list': merge_lists,
    'sentiment_list': merge_lists
})

# Display the result
df_reports_arguments

,filename,document_id,argument_list,sentiment_list
0,20140114_Needham_NVDA_NVDA-_Compelling_Technol...,512,"[Technology Trends, Market Expansion, Technolo...","[positive, negative, positive, positive, posit..."
1,20140116_Brean_Capital_SIG_SIG-_Some_Pyrite_Mi...,909,"[Market Share, Revenue, Growth, Revenue, Margi...","[positive, positive, positive, positive, negat..."
2,20140116_Brean_Capital_URBN_URBN-_Ups_-_Downs-...,717,"[Competition, Margins, Competition, Revenue, M...","[negative, negative, negative, negative, negat..."
3,20140122_Pivotal_Research_Group_GOOGL_GOOG-_Gr...,917,"[Margins, Long-term Growth, Investments, Margi...","[negative, negative, negative, negative, posit..."
4,20140124_Hilliard_Lyons_PG_Report_received_in_...,574,"[Revenue, Market Share, Margins, Earnings per ...","[positive, positive, negative, positive, posit..."
...,...,...,...,...
911,20221227_JP_Morgan_WFC_Wells_Fargo-_A_New_Surp...,793,"[Regulations, Growth, Regulations, Financial P...","[negative, negative, negative, negative, negat..."
912,20221228_JP_Morgan_UAA_Under_Armour-_Inc.-_Mgm...,908,"[Leadership, Growth, Market Share, Growth, Mar...","[positive, positive, negative, positive, posit..."
913,20221228_Wells_Fargo_FDX_FDX-_Good_News-_Bad_N...,642,"[Margins, Cost Management, Earnings Forecast, ...","[positive, neutral, negative, positive, neutra..."
914,20221229_JP_Morgan_MUR_Murphy-_The_Road_Ahead_...,22,"[Stock Performance, Cash Flow, Dividend Policy...","[positive, positive, positive, positive, negat..."


<div style="background-color: #181818; padding: 5px; border-radius: 20px; text-align: center;">
    <h2 style="font-size: 1.7em;color:#FBFAFA">Argument Mapping</h2>
</div>

In [1116]:
#df_arguments_sentiment.to_csv("full_list_arguments_3.5.csv")

In [1117]:
from collections import defaultdict, Counter
import pandas as pd

def aggregate_sentiment_counts(df, argument_col="argument_list", sentiment_col="sentiment_list"):

    # Initialize a defaultdict for counting sentiments
    sentiment_counts = defaultdict(lambda: Counter({'positive': 0, 'neutral': 0, 'negative': 0}))
    
    # Iterate over rows in the DataFrame
    for _, row in df.iterrows():
        argument_list = row[argument_col]
        sentiment_list = row[sentiment_col]
        
        # Ensure both are lists of the same length
        if isinstance(argument_list, list) and isinstance(sentiment_list, list) and len(argument_list) == len(sentiment_list):
            for argument, sentiment in zip(argument_list, sentiment_list):
                if isinstance(argument, str) and isinstance(sentiment, str):
                    sentiment_counts[argument][sentiment] += 1
                else:
                    print(f"Skipping invalid argument-sentiment pair: {argument}, {sentiment}")
    
    # Convert the counts to a DataFrame
    sentiment_overview_df = pd.DataFrame(sentiment_counts).T
    sentiment_overview_df.columns = ["positive", "neutral", "negative"]
    sentiment_overview_df.index.name = "Argument"
    
    # Reset the index for easier viewing
    sentiment_overview_df.reset_index(inplace=True)
    
    # Sort by positive, then negative, then neutral counts
    sentiment_overview_df.sort_values(by=["positive", "negative", "neutral"], ascending=False, inplace=True)
    
    return sentiment_overview_df

In [1118]:
# Assume df_paragraphs_arguments is your DataFrame
sentiment_overview_df = aggregate_sentiment_counts(df_paragraphs_arguments, 
                                                   argument_col="argument_list", 
                                                   sentiment_col="sentiment_list")

In [1119]:
print(f"Amounf of arguments after Mapping {len(sentiment_overview_df)}")

Amounf of arguments after Mapping 283


In [1120]:
sentiment_overview_df
sentiment_overview_df["total"] = sentiment_overview_df[["positive","negative","neutral"]].sum(axis=1)
sentiment_overview_df["relative_total"] = (sentiment_overview_df["total"] / sentiment_overview_df["total"].sum())
sentiment_overview_df = sentiment_overview_df.sort_values('total',ascending=False)
#sentiment_overview_df = sentiment_overview_df[sentiment_overview_df["total"] > 3 ]
sentiment_overview_df.head(10)
sentiment_overview_df.to_csv("sentiment_overview_part1.csv")


# Sentiment Analysis

In [1121]:
# Get dataframe for successful recommendations
df_sentiment_analysis = sentiment_overview_df

# Display the overview DataFrame
df_sentiment_analysis.reset_index(inplace=True)


In [1122]:
# Create Ratio fields
df_sentiment_analysis["argument_importance"] = round((df_sentiment_analysis["total"] / df_sentiment_analysis["total"].sum()),4)
df_sentiment_analysis["positive_ratio"] = round((df_sentiment_analysis["positive"] / df_sentiment_analysis["total"]),4)
df_sentiment_analysis["negative_ratio"] = round((df_sentiment_analysis["negative"] / df_sentiment_analysis["total"]),4)
df_sentiment_analysis["neutral_ratio"] = round((df_sentiment_analysis["neutral"] / df_sentiment_analysis["total"]),4)
df_sentiment_analysis["opinion_score"] = round((df_sentiment_analysis["positive"] / df_sentiment_analysis["total"]),4) - round((df_sentiment_analysis["negative"] / df_sentiment_analysis["total"]),4)

#df_sentiment_analysis["positive"] = df_sentiment_analysis["positive"].apply(lambda x: x.int())
df_sentiment_analysis = df_sentiment_analysis.sort_values(by=["total"],ascending=False)
df_sentiment_analysis["position"] = range(1,len(df_sentiment_analysis)+1)
df_sentiment_analysis.to_csv("sentiment_overview_part1.csv")

In [1123]:
df_success_evaluation_reports = pd.merge(df_reports_arguments,df_reports[["filename","tp_reached","rating","industry","adjusted_target_price"]],on="filename",how='left')

In [1124]:
df_argument_per_industry_rating = df_success_evaluation_reports

In [1125]:
df_argument_per_industry_rating

,filename,document_id,argument_list,sentiment_list,tp_reached,rating,industry,adjusted_target_price
0,20140114_Needham_NVDA_NVDA-_Compelling_Technol...,512,"[Technology Trends, Market Expansion, Technolo...","[positive, negative, positive, positive, posit...",False,hold,Technology,NaN
1,20140116_Brean_Capital_SIG_SIG-_Some_Pyrite_Mi...,909,"[Market Share, Revenue, Growth, Revenue, Margi...","[positive, positive, positive, positive, negat...",True,buy,Retail,88.00
2,20140116_Brean_Capital_URBN_URBN-_Ups_-_Downs-...,717,"[Competition, Margins, Competition, Revenue, M...","[negative, negative, negative, negative, negat...",False,hold,Retail,NaN
3,20140122_Pivotal_Research_Group_GOOGL_GOOG-_Gr...,917,"[Margins, Long-term Growth, Investments, Margi...","[negative, negative, negative, negative, posit...",True,hold,Communication Services,26.65
4,20140124_Hilliard_Lyons_PG_Report_received_in_...,574,"[Revenue, Market Share, Margins, Earnings per ...","[positive, positive, negative, positive, posit...",False,neutral,Consumer Staples,NaN
...,...,...,...,...,...,...,...,...
911,20221227_JP_Morgan_WFC_Wells_Fargo-_A_New_Surp...,793,"[Regulations, Growth, Regulations, Financial P...","[negative, negative, negative, negative, negat...",False,neutral,Financials,NaN
912,20221228_JP_Morgan_UAA_Under_Armour-_Inc.-_Mgm...,908,"[Leadership, Growth, Market Share, Growth, Mar...","[positive, positive, negative, positive, posit...",False,overweight,Clothing,NaN
913,20221228_Wells_Fargo_FDX_FDX-_Good_News-_Bad_N...,642,"[Margins, Cost Management, Earnings Forecast, ...","[positive, neutral, negative, positive, neutra...",False,equal weight,Industrials,185.00
914,20221229_JP_Morgan_MUR_Murphy-_The_Road_Ahead_...,22,"[Stock Performance, Cash Flow, Dividend Policy...","[positive, positive, positive, positive, negat...",False,overweight,Energy,56.00


In [1126]:
df_success_evaluation_reports = df_success_evaluation_reports[df_success_evaluation_reports["adjusted_target_price"].notna()]

In [1127]:
df_success_evaluation_reports

,filename,document_id,argument_list,sentiment_list,tp_reached,rating,industry,adjusted_target_price
1,20140116_Brean_Capital_SIG_SIG-_Some_Pyrite_Mi...,909,"[Market Share, Revenue, Growth, Revenue, Margi...","[positive, positive, positive, positive, negat...",True,buy,Retail,88.00
3,20140122_Pivotal_Research_Group_GOOGL_GOOG-_Gr...,917,"[Margins, Long-term Growth, Investments, Margi...","[negative, negative, negative, negative, posit...",True,hold,Communication Services,26.65
6,20140128_BGC_Partners_EBAY_BGC_EBAY_01_21.2014...,755,"[Revenue, Growth, Investments, Market Share, G...","[negative, positive, positive, positive, posit...",False,buy,Consumer Discretionary,59.00
8,20140131_BGC_Partners_MSFT_BGC_MSFT_01242014.pdf,603,"[Growth, Leadership, Technology Trends, Revenu...","[positive, positive, positive, positive, posit...",False,hold,Technology,35.00
9,20140203_BGC_Partners_AAPL_BGC_AAPL_01272014.pdf,362,"[Risk Management, Price/Earnings Ratio, Compet...","[neutral, neutral, negative, neutral, negative...",True,hold,Technology,19.64
...,...,...,...,...,...,...,...,...
907,20221220_Wells_Fargo_MRK_MRK-_MRNA_Cancer_Vacc...,70,"[Earnings, Market Expansion, Product Character...","[positive, positive, positive, positive, posit...",True,overweight,Healthcare,110.00
909,20221221_Wells_Fargo_ABT_ABT-_Exiting_the_Chin...,772,"[Market Share, Revenue, Revenue, Growth, Reven...","[negative, negative, negative, positive, posit...",False,overweight,Healthcare,140.00
910,20221223_Deutsche_Bank_ADBE_Adobe-_F4Q-_Solid_...,182,"[Financial Performance, Market Conditions, Lon...","[positive, positive, positive, neutral, neutra...",False,buy,Technology,430.00
913,20221228_Wells_Fargo_FDX_FDX-_Good_News-_Bad_N...,642,"[Margins, Cost Management, Earnings Forecast, ...","[positive, neutral, negative, positive, neutra...",False,equal weight,Industrials,185.00


<div style="background-color: #181818; padding: 5px; border-radius: 20px; text-align: center;">
    <h2 style="font-size: 1.7em;color:#FBFAFA">Successful Reports</h2>
</div>

In [1128]:
df_success_reports = df_success_evaluation_reports[df_success_evaluation_reports["tp_reached"]==True]

In [1129]:
df_success_reports

,filename,document_id,argument_list,sentiment_list,tp_reached,rating,industry,adjusted_target_price
1,20140116_Brean_Capital_SIG_SIG-_Some_Pyrite_Mi...,909,"[Market Share, Revenue, Growth, Revenue, Margi...","[positive, positive, positive, positive, negat...",True,buy,Retail,88.00
3,20140122_Pivotal_Research_Group_GOOGL_GOOG-_Gr...,917,"[Margins, Long-term Growth, Investments, Margi...","[negative, negative, negative, negative, posit...",True,hold,Communication Services,26.65
9,20140203_BGC_Partners_AAPL_BGC_AAPL_01272014.pdf,362,"[Risk Management, Price/Earnings Ratio, Compet...","[neutral, neutral, negative, neutral, negative...",True,hold,Technology,19.64
15,20140220_Needham_TRIP_TRIP-_4Q13_Post_Revising...,704,"[Revenue, Earnings per Share, Growth, Revenue,...","[positive, positive, positive, positive, posit...",True,buy,Travel,90.00
20,20140328_Needham_ABT_CSII-_Diamondback_Ready_t...,726,"[Market Share, Revenue, Market Expansion, Valu...","[positive, positive, positive, positive, posit...",True,buy,Healthcare,47.00
...,...,...,...,...,...,...,...,...
877,20221115_Wells_Fargo_COTY_COTY-_Not_So_Mass_Ev...,59,"[Financial Performance, Valuation, Revenue, Ma...","[positive, neutral, neutral, positive, positiv...",True,equal weight,Beauty,7.00
890,20221125_Deutsche_Bank_GM_General_Motors_Co-_E...,848,"[Growth, Revenue, Cost Management, Investments...","[positive, positive, negative, neutral, neutra...",True,hold,Automobile,35.00
897,20221208_Wells_Fargo_COST_COST-_More_Evidence_...,212,"[Growth, Revenue, Market Conditions, Growth, M...","[negative, negative, negative, negative, negat...",True,equal weight,Consumer Staples,490.00
899,20221209_Wells_Fargo_ULTA_ULTA-_Rarity_in_Reta...,737,"[Revenue, Margins, Long-term Growth, Margins, ...","[positive, neutral, negative, negative, positi...",True,equal weight,Beauty,450.00


In [1130]:
#Should I reassign the arguments here??

# Get dataframe for successful recommendations
df_success_reports = df_success_evaluation_reports[df_success_evaluation_reports["tp_reached"]==True]

# Initialize a dictionary to hold counts for each key and sentiment
sentiment_counts_success = defaultdict(lambda: Counter({'positive': 0, 'neutral': 0, 'negative': 0}))

def aggregate_sentiment_counts(df, argument_col="argument_list", sentiment_col="sentiment_list"):

    # Initialize a defaultdict for counting sentiments
    sentiment_counts = defaultdict(lambda: Counter({'positive': 0, 'neutral': 0, 'negative': 0}))
    
    # Iterate over rows in the DataFrame
    for _, row in df.iterrows():
        argument_list = row[argument_col]
        sentiment_list = row[sentiment_col]
        
        # Ensure both are lists of the same length
        if isinstance(argument_list, list) and isinstance(sentiment_list, list) and len(argument_list) == len(sentiment_list):
            for argument, sentiment in zip(argument_list, sentiment_list):
                if isinstance(argument, str) and isinstance(sentiment, str):
                    sentiment_counts[argument][sentiment] += 1
                else:
                    print(f"Skipping invalid argument-sentiment pair: {argument}, {sentiment}")
    
    # Convert the counts to a DataFrame
    df_sentiment_success = pd.DataFrame(sentiment_counts).T
    df_sentiment_success.columns = ["positive", "neutral", "negative"]
    df_sentiment_success.index.name = "Argument"
    
    # Reset the index for easier viewing
    df_sentiment_success.reset_index(inplace=True)
    
    # Sort by positive, then negative, then neutral counts
    df_sentiment_success.sort_values(by=["positive", "negative", "neutral"], ascending=False, inplace=True)
    
    return df_sentiment_success

# Assume df_paragraphs_arguments is your DataFrame
df_sentiment_success = aggregate_sentiment_counts(df_success_reports, 
                                                   argument_col="argument_list", 
                                                   sentiment_col="sentiment_list")



# Display the overview DataFrame
df_sentiment_success.reset_index(inplace=True)
df_sentiment_success["total"] = df_sentiment_success[["positive","negative","neutral"]].sum(axis=1)

In [1131]:
#df_sentiment_success = df_sentiment_success[df_sentiment_success["total"] > 3 ]
df_sentiment_success

,index,Argument,positive,neutral,negative,total
0,2,Growth,1284,43,187,1514
1,1,Revenue,960,115,239,1314
2,0,Market Share,475,28,92,595
3,3,Margins,391,50,255,696
4,11,Technology Trends,303,16,26,345
...,...,...,...,...,...,...
205,171,Tax Settlement,0,1,0,1
206,173,Securities,0,1,0,1
207,180,Executive Changes,0,1,0,1
208,182,Fees,0,1,0,1


In [1132]:
# Create Ratio fields
df_sentiment_success["argument_importance"] = round((df_sentiment_success["total"] / df_sentiment_success["total"].sum()),4)
df_sentiment_success["positive_ratio"] = round((df_sentiment_success["positive"] / df_sentiment_success["total"]),4)
df_sentiment_success["negative_ratio"] = round((df_sentiment_success["negative"] / df_sentiment_success["total"]),4)
df_sentiment_success["neutral_ratio"] = round((df_sentiment_success["neutral"] / df_sentiment_success["total"]),4)
df_sentiment_success["opinion_score"] = round((df_sentiment_success["positive"] / df_sentiment_success["total"]),4) - round((df_sentiment_success["negative"] / df_sentiment_success["total"]),4)

df_sentiment_success = df_sentiment_success.sort_values(by=["positive","negative","neutral"],ascending=False)
df_sentiment_success["position"] = range(1,len(df_sentiment_success)+1)
df_sentiment_success.head(30)

,index,Argument,positive,neutral,negative,total,argument_importance,positive_ratio,negative_ratio,neutral_ratio,opinion_score,position
0,2,Growth,1284,43,187,1514,0.1308,0.8481,0.1235,0.0284,0.7246,1
1,1,Revenue,960,115,239,1314,0.1135,0.7306,0.1819,0.0875,0.5487,2
2,0,Market Share,475,28,92,595,0.0514,0.7983,0.1546,0.0471,0.6437,3
3,3,Margins,391,50,255,696,0.0601,0.5618,0.3664,0.0718,0.1954,4
4,11,Technology Trends,303,16,26,345,0.0298,0.8783,0.0754,0.0464,0.8029,5
5,7,Investments,259,65,70,394,0.0340,0.6574,0.1777,0.1650,0.4797,6
6,25,Market Expansion,259,9,26,294,0.0254,0.8810,0.0884,0.0306,0.7926,7
7,33,Customer,211,26,32,269,0.0232,0.7844,0.1190,0.0967,0.6654,8
8,12,Cost Management,208,31,158,397,0.0343,0.5239,0.3980,0.0781,0.1259,9
9,21,Innovation,205,7,17,229,0.0198,0.8952,0.0742,0.0306,0.8210,10


<div style="background-color: #181818; padding: 5px; border-radius: 20px; text-align: center;">
    <h2 style="font-size: 1.7em;color:#FBFAFA">Not Successful Reports</h2>
</div>

In [1133]:
df_unsuccess_reports = df_success_evaluation_reports[df_success_evaluation_reports["tp_reached"]==False]

# Initialize a dictionary to hold counts for each key and sentiment
sentiment_counts_success = defaultdict(lambda: Counter({'positive': 0, 'neutral': 0, 'negative': 0}))

def aggregate_sentiment_counts(df, argument_col="argument_list", sentiment_col="sentiment_list"):

    # Initialize a defaultdict for counting sentiments
    sentiment_counts = defaultdict(lambda: Counter({'positive': 0, 'neutral': 0, 'negative': 0}))
    
    # Iterate over rows in the DataFrame
    for _, row in df.iterrows():
        argument_list = row[argument_col]
        sentiment_list = row[sentiment_col]
        
        # Ensure both are lists of the same length
        if isinstance(argument_list, list) and isinstance(sentiment_list, list) and len(argument_list) == len(sentiment_list):
            for argument, sentiment in zip(argument_list, sentiment_list):
                if isinstance(argument, str) and isinstance(sentiment, str):
                    sentiment_counts[argument][sentiment] += 1
                else:
                    print(f"Skipping invalid argument-sentiment pair: {argument}, {sentiment}")
    
    # Convert the counts to a DataFrame
    df_sentiment_unsuccess = pd.DataFrame(sentiment_counts).T
    df_sentiment_unsuccess.columns = ["positive", "neutral", "negative"]
    df_sentiment_unsuccess.index.name = "Argument"
    
    # Reset the index for easier viewing
    df_sentiment_unsuccess.reset_index(inplace=True)
    
    # Sort by positive, then negative, then neutral counts
    df_sentiment_unsuccess.sort_values(by=["positive", "negative", "neutral"], ascending=False, inplace=True)
    
    return df_sentiment_unsuccess

# Assume df_paragraphs_arguments is your DataFrame
df_sentiment_unsuccess = aggregate_sentiment_counts(df_unsuccess_reports, 
                                                   argument_col="argument_list", 
                                                   sentiment_col="sentiment_list")



# Display the overview DataFrame
df_sentiment_unsuccess.reset_index(inplace=True)
df_sentiment_unsuccess["total"] = df_sentiment_unsuccess[["positive","negative","neutral"]].sum(axis=1)

In [1134]:
# Create Ratio fields
#df_sentiment_unsuccess = df_sentiment_unsuccess[df_sentiment_unsuccess["total"] > 3 ]

df_sentiment_unsuccess["argument_importance"] = round((df_sentiment_unsuccess["total"] / df_sentiment_unsuccess["total"].sum()),4)
df_sentiment_unsuccess["positive_ratio"] = round((df_sentiment_unsuccess["positive"] / df_sentiment_unsuccess["total"]),4)
df_sentiment_unsuccess["negative_ratio"] = round((df_sentiment_unsuccess["negative"] / df_sentiment_unsuccess["total"]),4)
df_sentiment_unsuccess["neutral_ratio"] = round((df_sentiment_unsuccess["neutral"] / df_sentiment_unsuccess["total"]),4)
df_sentiment_unsuccess["opinion_score"] = round((df_sentiment_unsuccess["positive"] / df_sentiment_unsuccess["total"]),4) - round((df_sentiment_unsuccess["negative"] / df_sentiment_unsuccess["total"]),4)

df_sentiment_unsuccess = df_sentiment_unsuccess.sort_values(by=["positive","negative","neutral"],ascending=False)
df_sentiment_unsuccess["position"] = range(1,len(df_sentiment_unsuccess)+1)

df_sentiment_unsuccess.head(20)

,index,Argument,positive,neutral,negative,total,argument_importance,positive_ratio,negative_ratio,neutral_ratio,opinion_score,position
0,1,Growth,916,30,121,1067,0.1171,0.8585,0.1134,0.0281,0.7451,1
1,0,Revenue,789,80,201,1070,0.1174,0.7374,0.1879,0.0748,0.5495,2
2,3,Market Share,361,10,70,441,0.0484,0.8186,0.1587,0.0227,0.6599,3
3,14,Margins,321,28,174,523,0.0574,0.6138,0.3327,0.0535,0.2811,4
4,2,Investments,244,53,58,355,0.0390,0.6873,0.1634,0.1493,0.5239,5
5,17,Market Expansion,197,4,20,221,0.0243,0.8914,0.0905,0.0181,0.8009,6
6,13,Technology Trends,174,8,16,198,0.0217,0.8788,0.0808,0.0404,0.7980,7
7,6,Cash Flow,170,15,44,229,0.0251,0.7424,0.1921,0.0655,0.5503,8
8,27,Customer,166,17,20,203,0.0223,0.8177,0.0985,0.0837,0.7192,9
9,21,Innovation,162,2,7,171,0.0188,0.9474,0.0409,0.0117,0.9065,10


<div style="background-color: #181818; padding: 5px; border-radius: 20px; text-align: center;">
    <h2 style="font-size: 1.7em;color:#FBFAFA">Comparison of Success</h2>
</div>

https://stackoverflow.com/questions/52783391/how-to-merge-dictionaries-of-a-pandas-dataframe-when-grouping-by-rows

In [1135]:
# Merge the two DataFrames on 'Argument' to align rows
merged_df = pd.merge(df_sentiment_success, df_sentiment_unsuccess, on='Argument', suffixes=('_df1', '_df2'),how="outer")
merged_df = merged_df.fillna(0)
merged_df["average_total"] = np.where(
    (merged_df["total_df1"] + merged_df["total_df2"]) / 2 == 0,
    np.maximum(merged_df["total_df1"], merged_df["total_df2"]),
    (merged_df["total_df1"] + merged_df["total_df2"]) / 2
)

In [1136]:
# Calculate differences for all numeric columns (excluding 'Argument')
difference_df = merged_df[['Argument',"average_total"]].copy()  # Start with the 'Argument' column
columns_to_compare = [col for col in df_sentiment_success.columns if col != 'Argument' and col != 'average_total']  # Columns to compute differences for

for col in columns_to_compare:
    difference_df[col + '_difference'] = merged_df[col + '_df1'] - merged_df[col + '_df2']

In [1137]:
difference_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260 entries, 0 to 259
Data columns (total 13 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Argument                        260 non-null    object 
 1   average_total                   260 non-null    float64
 2   index_difference                260 non-null    float64
 3   positive_difference             260 non-null    float64
 4   neutral_difference              260 non-null    float64
 5   negative_difference             260 non-null    float64
 6   total_difference                260 non-null    float64
 7   argument_importance_difference  260 non-null    float64
 8   positive_ratio_difference       260 non-null    float64
 9   negative_ratio_difference       260 non-null    float64
 10  neutral_ratio_difference        260 non-null    float64
 11  opinion_score_difference        260 non-null    float64
 12  position_difference             260 

In [1138]:
difference_df.to_csv("difference_testing.csv")

<div style="background-color: #181818; padding: 5px; border-radius: 20px; text-align: center;">
    <h2 style="font-size: 1.7em;color:#FBFAFA">Statistical Significance</h2>
</div>
https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.chisquare.html

In [1139]:
import numpy as np

def cohens_d(group1, group2):
    # Means of the groups
    mean1, mean2 = np.mean(group1), np.mean(group2)
    
    # Standard deviations of the groups
    std1, std2 = np.std(group1, ddof=1), np.std(group2, ddof=1)
    
    # Sizes of the groups
    n1, n2 = len(group1), len(group2)
    
    # Pooled standard deviation
    pooled_std = np.sqrt(((n1 - 1) * std1**2 + (n2 - 1) * std2**2) / (n1 + n2 - 2))
    
    # Compute Cohen's d
    return (mean1 - mean2) / pooled_std

In [1140]:
## I think this is not correct!

import numpy as np

# Extract total success and unsuccess counts
success_counts = merged_df["total_df1"].tolist()
unsuccess_counts = merged_df["total_df2"].tolist()

cohens_d_values = []

counter = 0

for success, unsuccess in zip(success_counts, unsuccess_counts):
    # Means are the counts themselves
    counter +=1
    mean_success = success
    mean_unsuccess = unsuccess
    
    # Approximate standard deviations (for count data, use sqrt(count))
    std_success = np.sqrt(success)  # Estimated standard deviation for success
    std_unsuccess = np.sqrt(unsuccess)  # Estimated standard deviation for unsuccess
    
    # Compute pooled standard deviation
    pooled_std = np.sqrt(((std_success**2 + std_unsuccess**2) / 2))
    
    # Compute Cohen's d
    d = (mean_success - mean_unsuccess) / pooled_std
    cohens_d_values.append(d)

# Add Cohen's d values to the DataFrame
merged_df["cohen_d"] = cohens_d_values

# Display results
#print(merged_df[["Argument", "cohen_d"]])

In [1141]:
import pandas as pd
import scipy.stats as stats
from statsmodels.stats.proportion import proportions_ztest
import pingouin as pg


# Step 1: Chi-square test for overall usage
merged_df["relative_usage_df1"] = merged_df["total_df1"] / merged_df["total_df1"].sum()
merged_df["relative_usage_df2"] = merged_df["total_df2"] / merged_df["total_df2"].sum()

# Create lists to store test results
statistical_significance_values = []

# Loop through each argument
for _, row in merged_df.iterrows():
    argument = row["Argument"]

    successful_count = row["total_df1"]
    not_successful_count = row["total_df2"]
    other_successful_count = sum(merged_df["total_df1"]) - successful_count
    other_not_successful_count = sum(merged_df["total_df2"]) - not_successful_count

    contingency_table = [[successful_count, other_successful_count],
                     [not_successful_count, other_not_successful_count]]

    chi2_stat, p_total, _, _ = stats.chi2_contingency(contingency_table)

    # Step 2: Proportion z-test for relative usage
    count = [row["relative_usage_df1"] * sum(merged_df["total_df1"]),
             row["relative_usage_df2"] * sum(merged_df["total_df2"])]
    nobs = [sum(merged_df["total_df1"]), sum(merged_df["total_df2"])]
    z_stat, p_relative = proportions_ztest(count, nobs)

    # Compute Cohen's d for each argument
    # Store results
    statistical_significance_values.append({
        "Argument": argument,
        "p_total": p_total,
        "p_relative": p_relative,
        "z_stat":z_stat,
        "success_count":row["total_df1"],
        "unsuccess_count":row["total_df2"],
        "cohen_d":row["cohen_d"]
    })

# Create a DataFrame for results
df_statistical_significance = pd.DataFrame(statistical_significance_values)

# Display the results
df_statistical_significance.head(30)
df_statistical_significance.sort_values('cohen_d',ascending=False).head(50)

,Argument,p_total,p_relative,z_stat,success_count,unsuccess_count,cohen_d
109,Growth,0.003259,0.003043,2.963373,1514.0,1067.0,12.443098
239,Technology Trends,0.000365,0.000308,3.608219,345.0,198.0,8.921388
61,E-commerce,0.000127,0.000091,3.914189,107.0,42.0,7.530698
198,Revenue,0.395931,0.383841,-0.870840,1314.0,1070.0,7.067270
141,Margins,0.423227,0.406216,0.830571,696.0,523.0,7.007438
148,Market Share,0.340171,0.324166,0.985933,595.0,441.0,6.766372
70,Earnings per Share,0.053257,0.047788,1.979247,291.0,191.0,6.441566
250,Valuation,0.055506,0.048661,1.971549,197.0,124.0,5.762160
177,Product Characteristics,0.012169,0.008810,2.619321,57.0,24.0,5.185450
19,Competition,0.710450,0.681359,0.410610,384.0,293.0,4.946090


## Create new columns
1. Add TP_Reached_as one column
2. Add total_as_arguments one column

# 2. Replace similar arguments with broader category

# 3. Calculate number of positive / negative / neutral Arguments

# Last Step: Merge Dictionaries

# Report Analysis

<div style="background-color: #181818; padding: 5px; border-radius: 20px; text-align: center;">
    <h2 style="font-size: 1.7em;color:#FBFAFA">Report Analysis</h2>
</div>

In [1142]:
from collections import Counter

# Function to get top 3 most frequent arguments
def get_top_arguments(arg_list, n):
    counter = Counter(arg_list)  # Count occurrences of each argument
    most_common = [item[0] for item in counter.most_common(n)]  # Extract top `n` arguments
    return most_common

# Apply the function to argument_list and create new columns
df_reports_arguments['1st_most_used_argument'] = df_reports_arguments['argument_list'].apply(lambda x: get_top_arguments(x, 1)[0] if len(get_top_arguments(x, 1)) > 0 else None)
df_reports_arguments['2nd_most_used_argument'] = df_reports_arguments['argument_list'].apply(lambda x: get_top_arguments(x, 2)[1] if len(get_top_arguments(x, 2)) > 1 else None)
df_reports_arguments['3rd_most_used_argument'] = df_reports_arguments['argument_list'].apply(lambda x: get_top_arguments(x, 3)[2] if len(get_top_arguments(x, 3)) > 2 else None)

In [1143]:
df_final_reports = pd.merge(df_reports,df_reports_arguments,on='filename',how='inner')
df_final_reports['year'] = pd.to_datetime(df_final_reports['date']).dt.year
df_final_reports["opinion_score"] = round((df_final_reports["positive_arguments"] / df_final_reports["total_arguments"]),2) - round((df_final_reports["negative_arguments"] / df_final_reports["total_arguments"]),2)

In [1144]:
pd.reset_option('display.max_colwidth', None)  # Don't truncate column content
df_final_reports

,document_id_x,filename,date,provider,ticker,company_name,industry,target_price,rating,start price,...,tp_reached,total_arguments,document_id_y,argument_list,sentiment_list,1st_most_used_argument,2nd_most_used_argument,3rd_most_used_argument,year,opinion_score
0,1,20161018_Needham_META_Facebook-_3Q16_Preview_R...,2016-10-11,Needham,META,Meta Platforms Inc.,Communication Services,150.0,buy,128.88,...,True,20,1,"[Revenue, Earnings per Share, Revenue, Revenue...","[positive, positive, positive, positive, posit...",Revenue,Growth,Earnings per Share,2016,0.60
1,2,20200807_Wells_Fargo_META_FB-_2.5B_Person_Plat...,2020-07-31,Wells Fargo,META,Meta Platforms Inc.,Communication Services,300.0,overweight,253.67,...,True,29,2,"[Revenue, Growth, Cost Management, Market Cond...","[positive, neutral, positive, negative, positi...",Revenue,Growth,Market Share,2020,0.35
2,3,20200918_Barclays_GM_General_Motors-_Time_to_t...,2020-09-14,Barclays,GM,General Motors Co.,Automobile,39.0,overweight,31.18,...,True,22,3,"[Growth, Innovation, Market Expansion, Growth,...","[positive, positive, positive, positive, posit...",Innovation,Growth,Market Expansion,2020,0.95
3,4,20200722_Barclays_BKNG_Booking_Holdings_Inc.-_...,2020-07-16,Barclays,BKNG,Booking Holdings Inc,Travel,1950.0,overweight,1746.79,...,True,14,4,"[Revenue, Margins, Market Conditions, Market E...","[negative, negative, positive, positive, posit...",Market Conditions,Revenue,Margins,2020,0.14
4,5,20211104_Deutsche_Bank_X_US_Steel-_3Q21_EBITDA...,2021-10-28,Deutsche Bank,X,United States Steel,Materials,50.0,buy,23.38,...,False,18,5,"[Growth, Investments, Financial Performance, F...","[positive, positive, positive, positive, neutr...",Growth,Financial Performance,Cash Flow,2021,0.72
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
911,913,20150522_Gilford_Securities_Inc_FL_Report_rece...,2015-05-22,Gilford Securities Inc,FL,Foot Locker,Clothing,NaN,buy,63.46,...,False,18,913,"[Growth, Earnings per Share, Dividend Policy, ...","[positive, positive, positive, positive, posit...",Growth,Earnings per Share,Dividend Policy,2015,0.56
912,914,20200526_Wells_Fargo_HD_HD-_Q2_Rollover_Falls_...,2020-05-19,Wells Fargo,HD,Home Depot Inc.,Consumer Discretionary,270.0,overweight,238.10,...,True,19,914,"[Market Share, Revenue, Sustainability, Financ...","[positive, positive, positive, positive, neutr...",Market Share,Growth,Market Conditions,2020,0.63
913,915,20220125_Deutsche_Bank_WFC_Wells_Fargo-_More_C...,2022-01-17,Deutsche Bank,WFC,Wells Fargo & Co.,Financials,55.0,buy,58.06,...,True,31,915,"[Earnings Forecast, Economic Conditions, Cost ...","[positive, positive, neutral, positive, positi...",Cost Management,Revenue,Economic Conditions,2022,0.20
914,916,20200212_Barclays_GM_General_Motors-_Waiting_p...,2020-02-06,Barclays,GM,General Motors Co.,Automobile,44.0,overweight,34.33,...,True,59,916,"[Valuation, Revenue, Growth, Market Cap, Earni...","[positive, positive, positive, positive, posit...",Growth,Revenue,Valuation,2020,0.44


# Opinion Score per Year & Industry

In [1145]:
df_opn_year = df_final_reports.groupby("year").agg(
    opinion_score =('opinion_score','mean'),
    total_reports =('filename','count')
)
df_opn_year

,opinion_score,total_reports
year,,
2014,0.302727,55
2015,0.387037,54
2016,0.394762,42
2017,0.376379,58
2018,0.216102,59
2019,0.374035,57
2020,0.291141,184
2021,0.350055,184
2022,0.284529,223


In [1146]:
df_opn_rating = df_final_reports.groupby("rating").agg(
    opinion_score =('opinion_score','mean'),
    total_reports =('filename','count')
)
df_opn_rating.sort_values('opinion_score',ascending=False)

,opinion_score,total_reports
rating,,
outperform,0.556667,3
buy,0.465812,235
overweight,0.437013,308
underperform,0.426364,11
equal weight,0.145890,73
hold,0.141189,143
neutral,0.124815,81
none,0.096471,17
underweight,0.003158,38


In [1147]:
df_opn_industry = df_final_reports.groupby("industry").agg(
    opinion_score =('opinion_score','mean'),
    total_reports =('filename','count')
)
df_opn_industry

,opinion_score,total_reports
industry,,
Automobile,0.301304,46
Beauty,0.297000,30
Clothing,0.294595,74
Communication Services,0.163939,66
Consumer Discretionary,0.425068,73
Consumer Staples,0.339388,49
Energy,0.255870,46
Financials,0.363438,96
Food,0.498939,66


In [1148]:
df_opn_industry_year = df_final_reports.groupby(["industry","year"]).agg(
    opinion_score =('opinion_score','mean'),
    total_reports =('filename','count')
)
df_opn_industry_year

opinion_score  total_reports
industry   year                              
Automobile 2014   3.000000e-02              3
           2015   7.000000e-01              1
           2018   5.000000e-03              2
           2019   2.350000e-01              2
           2020   5.514286e-01              7
...                        ...            ...
Travel     2018  -1.600000e-01              1
           2019  -3.700743e-17              3
           2020   1.410000e-01             10
           2021   9.900000e-02             10
           2022   1.810000e-01             10

[119 rows x 2 columns]

In [1149]:
df_final_reports

,document_id_x,filename,date,provider,ticker,company_name,industry,target_price,rating,start price,...,tp_reached,total_arguments,document_id_y,argument_list,sentiment_list,1st_most_used_argument,2nd_most_used_argument,3rd_most_used_argument,year,opinion_score
0,1,20161018_Needham_META_Facebook-_3Q16_Preview_R...,2016-10-11,Needham,META,Meta Platforms Inc.,Communication Services,150.0,buy,128.88,...,True,20,1,"[Revenue, Earnings per Share, Revenue, Revenue...","[positive, positive, positive, positive, posit...",Revenue,Growth,Earnings per Share,2016,0.60
1,2,20200807_Wells_Fargo_META_FB-_2.5B_Person_Plat...,2020-07-31,Wells Fargo,META,Meta Platforms Inc.,Communication Services,300.0,overweight,253.67,...,True,29,2,"[Revenue, Growth, Cost Management, Market Cond...","[positive, neutral, positive, negative, positi...",Revenue,Growth,Market Share,2020,0.35
2,3,20200918_Barclays_GM_General_Motors-_Time_to_t...,2020-09-14,Barclays,GM,General Motors Co.,Automobile,39.0,overweight,31.18,...,True,22,3,"[Growth, Innovation, Market Expansion, Growth,...","[positive, positive, positive, positive, posit...",Innovation,Growth,Market Expansion,2020,0.95
3,4,20200722_Barclays_BKNG_Booking_Holdings_Inc.-_...,2020-07-16,Barclays,BKNG,Booking Holdings Inc,Travel,1950.0,overweight,1746.79,...,True,14,4,"[Revenue, Margins, Market Conditions, Market E...","[negative, negative, positive, positive, posit...",Market Conditions,Revenue,Margins,2020,0.14
4,5,20211104_Deutsche_Bank_X_US_Steel-_3Q21_EBITDA...,2021-10-28,Deutsche Bank,X,United States Steel,Materials,50.0,buy,23.38,...,False,18,5,"[Growth, Investments, Financial Performance, F...","[positive, positive, positive, positive, neutr...",Growth,Financial Performance,Cash Flow,2021,0.72
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
911,913,20150522_Gilford_Securities_Inc_FL_Report_rece...,2015-05-22,Gilford Securities Inc,FL,Foot Locker,Clothing,NaN,buy,63.46,...,False,18,913,"[Growth, Earnings per Share, Dividend Policy, ...","[positive, positive, positive, positive, posit...",Growth,Earnings per Share,Dividend Policy,2015,0.56
912,914,20200526_Wells_Fargo_HD_HD-_Q2_Rollover_Falls_...,2020-05-19,Wells Fargo,HD,Home Depot Inc.,Consumer Discretionary,270.0,overweight,238.10,...,True,19,914,"[Market Share, Revenue, Sustainability, Financ...","[positive, positive, positive, positive, neutr...",Market Share,Growth,Market Conditions,2020,0.63
913,915,20220125_Deutsche_Bank_WFC_Wells_Fargo-_More_C...,2022-01-17,Deutsche Bank,WFC,Wells Fargo & Co.,Financials,55.0,buy,58.06,...,True,31,915,"[Earnings Forecast, Economic Conditions, Cost ...","[positive, positive, neutral, positive, positi...",Cost Management,Revenue,Economic Conditions,2022,0.20
914,916,20200212_Barclays_GM_General_Motors-_Waiting_p...,2020-02-06,Barclays,GM,General Motors Co.,Automobile,44.0,overweight,34.33,...,True,59,916,"[Valuation, Revenue, Growth, Market Cap, Earni...","[positive, positive, positive, positive, posit...",Growth,Revenue,Valuation,2020,0.44


In [1150]:
#df_final_reports.to_csv("final_reports_data.csv")

<div style="background-color: #181818; padding: 5px; border-radius: 20px; text-align: center;">
    <h2 style="font-size: 1.7em;color:#FBFAFA">Argument usage per unique Report</h2>
</div>

In [1151]:
# Count how many reports mention each argument at least once
def count_reports_with_argument(argument, df):
    return df['argument_list'].apply(lambda x: argument in x).sum()

sentiment_overview_df['part_of_reports_absolute'] = sentiment_overview_df['Argument'].apply(
    lambda arg: count_reports_with_argument(arg, df_reports_arguments)
)

In [1152]:
total_amount_of_reports = 917

df_grouped_arguments = sentiment_overview_df.sort_values("part_of_reports_absolute",ascending=False)
df_grouped_arguments["part_of_reports_relative"] = df_grouped_arguments["part_of_reports_absolute"] / total_amount_of_reports

In [1153]:
df_grouped_arguments.to_csv("argument_in_reports_analysis.csv")

<div style="background-color: #181818; padding: 5px; border-radius: 20px; text-align: center;">
    <h2 style="font-size: 1.7em;color:#FBFAFA">Industry Analysis</h2>
</div>

In [1154]:
df_argument_per_industry_rating

,filename,document_id,argument_list,sentiment_list,tp_reached,rating,industry,adjusted_target_price
0,20140114_Needham_NVDA_NVDA-_Compelling_Technol...,512,"[Technology Trends, Market Expansion, Technolo...","[positive, negative, positive, positive, posit...",False,hold,Technology,NaN
1,20140116_Brean_Capital_SIG_SIG-_Some_Pyrite_Mi...,909,"[Market Share, Revenue, Growth, Revenue, Margi...","[positive, positive, positive, positive, negat...",True,buy,Retail,88.00
2,20140116_Brean_Capital_URBN_URBN-_Ups_-_Downs-...,717,"[Competition, Margins, Competition, Revenue, M...","[negative, negative, negative, negative, negat...",False,hold,Retail,NaN
3,20140122_Pivotal_Research_Group_GOOGL_GOOG-_Gr...,917,"[Margins, Long-term Growth, Investments, Margi...","[negative, negative, negative, negative, posit...",True,hold,Communication Services,26.65
4,20140124_Hilliard_Lyons_PG_Report_received_in_...,574,"[Revenue, Market Share, Margins, Earnings per ...","[positive, positive, negative, positive, posit...",False,neutral,Consumer Staples,NaN
...,...,...,...,...,...,...,...,...
911,20221227_JP_Morgan_WFC_Wells_Fargo-_A_New_Surp...,793,"[Regulations, Growth, Regulations, Financial P...","[negative, negative, negative, negative, negat...",False,neutral,Financials,NaN
912,20221228_JP_Morgan_UAA_Under_Armour-_Inc.-_Mgm...,908,"[Leadership, Growth, Market Share, Growth, Mar...","[positive, positive, negative, positive, posit...",False,overweight,Clothing,NaN
913,20221228_Wells_Fargo_FDX_FDX-_Good_News-_Bad_N...,642,"[Margins, Cost Management, Earnings Forecast, ...","[positive, neutral, negative, positive, neutra...",False,equal weight,Industrials,185.00
914,20221229_JP_Morgan_MUR_Murphy-_The_Road_Ahead_...,22,"[Stock Performance, Cash Flow, Dividend Policy...","[positive, positive, positive, positive, negat...",False,overweight,Energy,56.00


In [1176]:
# Explode the argument_list so each argument has its own row
df_exploded = df_argument_per_industry_rating.explode('argument_list')

# Create a pivot table to count the occurrences of each argument per industry
pivot_table = pd.pivot_table(
    df_exploded,
    index='argument_list',
    columns='industry',
    aggfunc='size',
    fill_value=0
)
#pivot_table.to_csv("argument_per_rating.csv")

In [1177]:
pivot_table

industry,Automobile,Beauty,Clothing,Communication Services,Consumer Discretionary,Consumer Staples,Energy,Financials,Food,Healthcare,Industrials,Materials,Retail,Technology,Travel
argument_list,,,,,,,,,,,,,,,
API Integration,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
Accounting Practices,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0
Ad ARPU,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
Apparel Recovery,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
Asset-Light Models,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Volume Growth,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0
Volume Leverage,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0
Workflow Integration,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [1156]:
list_of_industries = [
    "Communication Services",
    "Automobile",
    "Travel",
    "Materials",
    "Healthcare",
    "Consumer Discretionary",
    "Energy",
    "Technology",
    "Industrials",
    "Retail",
    "Consumer Staples",
    "Beauty",
    "Financials",
    "Clothing",
    "Food",
]

df_industry_analysis = df_grouped_arguments.drop(columns=["positive","negative","neutral","part_of_reports_absolute","part_of_reports_relative"])

# Initialize columns for each industry
for industry in list_of_industries:
    df_industry_analysis[f'{industry}_absolute'] = 0
    df_industry_analysis[f'{industry}_relative'] = 0.0

# Count absolute and relative mentions per industry
for industry in list_of_industries:
    # Filter reports for the current industry
    industry_reports = df_final_reports[df_final_reports['industry'] == industry]
    total_reports_for_industry = len(industry_reports)

    if total_reports_for_industry > 0:  # Avoid division by zero
        df_industry_analysis[f'{industry}_absolute'] = df_industry_analysis['Argument'].apply(
            lambda arg: industry_reports['argument_list'].apply(lambda x: arg in x).sum()
        )
        df_industry_analysis[f'{industry}_relative'] = round((df_industry_analysis[f'{industry}_absolute'] / total_reports_for_industry),3)

In [1157]:
df_industry_analysis.head(40)

,index,Argument,total,relative_total,argument_importance,positive_ratio,negative_ratio,neutral_ratio,opinion_score,Communication Services_absolute,...,Consumer Staples_absolute,Consumer Staples_relative,Beauty_absolute,Beauty_relative,Financials_absolute,Financials_relative,Clothing_absolute,Clothing_relative,Food_absolute,Food_relative
0,6,Growth,3125,0.125633,0.1256,0.8387,0.1293,0.0320,0.7094,56,...,43,0.878,26,0.867,72,0.750,73,0.986,64,0.970
1,7,Revenue,2967,0.119281,0.1193,0.7085,0.1999,0.0917,0.5086,64,...,46,0.939,24,0.800,88,0.917,59,0.797,61,0.924
2,8,Margins,1503,0.060425,0.0604,0.5655,0.3653,0.0692,0.2002,41,...,34,0.694,25,0.833,42,0.438,61,0.824,51,0.773
4,9,Market Conditions,1116,0.044866,0.0449,0.3530,0.4928,0.1541,-0.1398,35,...,33,0.673,25,0.833,62,0.646,51,0.689,42,0.636
3,5,Market Share,1174,0.047198,0.0472,0.7990,0.1610,0.0400,0.6380,54,...,32,0.653,17,0.567,49,0.510,41,0.554,35,0.530
5,11,Investments,924,0.037147,0.0371,0.6656,0.1829,0.1515,0.4827,43,...,19,0.388,13,0.433,50,0.521,26,0.351,27,0.409
6,15,Cost Management,878,0.035298,0.0353,0.5091,0.4032,0.0877,0.1059,35,...,29,0.592,18,0.600,56,0.583,34,0.459,34,0.515
7,4,Competition,792,0.031840,0.0318,0.1780,0.7462,0.0758,-0.5682,51,...,24,0.490,11,0.367,34,0.354,40,0.541,17,0.258
8,13,Earnings per Share,702,0.028222,0.0282,0.6439,0.1752,0.1809,0.4687,28,...,19,0.388,10,0.333,36,0.375,35,0.473,30,0.455
11,36,Financial Performance,597,0.024001,0.0240,0.6181,0.2931,0.0888,0.3250,24,...,15,0.306,10,0.333,48,0.500,24,0.324,22,0.333


<div style="background-color: #181818; padding: 5px; border-radius: 20px; text-align: center;">
    <h2 style="font-size: 1.7em;color:#FBFAFA">Rating Analysis</h2>
</div>

In [1158]:
list_of_ratings = [
    "buy",
    "overweight",
    "neutral",
    "underweight",
    "hold",
    "equal weight",
    "underperform",
    "none",
    "outperform",
    "sell"
]

df_rating_analysis = df_grouped_arguments.drop(columns=["positive","negative","neutral","part_of_reports_absolute","part_of_reports_relative"])

# Initialize columns for each rating
for rating in list_of_ratings:
    df_rating_analysis[f'{rating}_absolute'] = 0
    df_rating_analysis[f'{rating}_relative'] = 0.0

# Count absolute and relative mentions per rating
for rating in list_of_ratings:
    # Filter reports for the current rating
    rating_reports = df_final_reports[df_final_reports['rating'] == rating]
    total_reports_for_rating = len(rating_reports)

    if total_reports_for_rating > 0:  # Avoid division by zero
        df_rating_analysis[f'{rating}_absolute'] = df_rating_analysis['Argument'].apply(
            lambda arg: rating_reports['argument_list'].apply(lambda x: arg in x).sum()
        )
        df_rating_analysis[f'{rating}_relative'] = round((df_rating_analysis[f'{rating}_absolute'] / total_reports_for_rating),3)

In [1159]:
df_rating_analysis.to_csv("rating_argument_analysis.csv")

<div style="background-color: #181818; padding: 5px; border-radius: 20px; text-align: center;">
    <h2 style="font-size: 1.7em;color:#FBFAFA">Year Analysis</h2>
</div>

In [1160]:
df_year_analysis = df_grouped_arguments.drop(columns=["positive","negative","neutral","part_of_reports_absolute","part_of_reports_relative"])

In [1161]:
df_year_analysis

,index,Argument,total,relative_total,argument_importance,positive_ratio,negative_ratio,neutral_ratio,opinion_score
0,6,Growth,3125,0.125633,0.1256,0.8387,0.1293,0.0320,0.7094
1,7,Revenue,2967,0.119281,0.1193,0.7085,0.1999,0.0917,0.5086
2,8,Margins,1503,0.060425,0.0604,0.5655,0.3653,0.0692,0.2002
4,9,Market Conditions,1116,0.044866,0.0449,0.3530,0.4928,0.1541,-0.1398
3,5,Market Share,1174,0.047198,0.0472,0.7990,0.1610,0.0400,0.6380
...,...,...,...,...,...,...,...,...,...
213,195,Earnings Call Focus,1,0.000040,0.0000,0.0000,0.0000,1.0000,0.0000
212,196,GPU Supply Outlook,1,0.000040,0.0000,0.0000,0.0000,1.0000,0.0000
211,198,Crypto Demand Impact,1,0.000040,0.0000,0.0000,0.0000,1.0000,0.0000
210,200,Crypto Volatility,1,0.000040,0.0000,0.0000,0.0000,1.0000,0.0000


In [1162]:
years = list(range(2014, 2023))

# Initialize columns for each year
for year in years:
    df_year_analysis[f'{year}_absolute'] = 0
    df_year_analysis[f'{year}_relative'] = 0.0

# Count absolute and relative mentions per year
for year in years:
    # Filter reports for the current year
    year_reports = df_final_reports[df_final_reports['year'] == year]
    total_reports_for_year = len(year_reports)

    if total_reports_for_year > 0:  # Avoid division by zero
        df_year_analysis[f'{year}_absolute'] = df_year_analysis['Argument'].apply(
            lambda arg: year_reports['argument_list'].apply(lambda x: arg in x).sum()
        )
        df_year_analysis[f'{year}_relative'] = round((df_year_analysis[f'{year}_absolute'] / total_reports_for_year),3)

In [1163]:
# Calculate standard deviation and variance for the relative values
relative_columns = [f'{year}_relative' for year in range(2014, 2023)]

df_year_analysis['relative_std_dev'] = df_year_analysis[relative_columns].std(axis=1)
df_year_analysis['relative_variance'] = df_year_analysis[relative_columns].var(axis=1)

In [1164]:
df_year_analysis.head(40)

,index,Argument,total,relative_total,argument_importance,positive_ratio,negative_ratio,neutral_ratio,opinion_score,2014_absolute,...,2019_absolute,2019_relative,2020_absolute,2020_relative,2021_absolute,2021_relative,2022_absolute,2022_relative,relative_std_dev,relative_variance
0,6,Growth,3125,0.125633,0.1256,0.8387,0.1293,0.0320,0.7094,51,...,54,0.947,151,0.821,156,0.848,170,0.762,0.064257,0.004129
1,7,Revenue,2967,0.119281,0.1193,0.7085,0.1999,0.0917,0.5086,48,...,49,0.860,157,0.853,153,0.832,180,0.807,0.031327,0.000981
2,8,Margins,1503,0.060425,0.0604,0.5655,0.3653,0.0692,0.2002,38,...,33,0.579,124,0.674,117,0.636,145,0.650,0.054505,0.002971
4,9,Market Conditions,1116,0.044866,0.0449,0.3530,0.4928,0.1541,-0.1398,39,...,23,0.404,127,0.690,99,0.538,150,0.673,0.111276,0.012382
3,5,Market Share,1174,0.047198,0.0472,0.7990,0.1610,0.0400,0.6380,37,...,29,0.509,98,0.533,96,0.522,111,0.498,0.071214,0.005071
5,11,Investments,924,0.037147,0.0371,0.6656,0.1829,0.1515,0.4827,33,...,31,0.544,79,0.429,83,0.451,111,0.498,0.057630,0.003321
6,15,Cost Management,878,0.035298,0.0353,0.5091,0.4032,0.0877,0.1059,26,...,23,0.404,100,0.543,92,0.500,104,0.466,0.068691,0.004719
7,4,Competition,792,0.031840,0.0318,0.1780,0.7462,0.0758,-0.5682,41,...,26,0.456,77,0.418,68,0.370,69,0.309,0.136604,0.018661
8,13,Earnings per Share,702,0.028222,0.0282,0.6439,0.1752,0.1809,0.4687,37,...,25,0.439,67,0.364,53,0.288,35,0.157,0.207173,0.042921
11,36,Financial Performance,597,0.024001,0.0240,0.6181,0.2931,0.0888,0.3250,14,...,22,0.386,73,0.397,65,0.353,89,0.399,0.071803,0.005156


<div style="background-color: #181818; padding: 5px; border-radius: 20px; text-align: center;">
    <h2 style="font-size: 1.7em;color:#FBFAFA">Target Price Analysis</h2>
</div>

df_reports

In [1165]:
df_reports["difference_tp_start_price"] = df_reports["adjusted_target_price"] - df_reports["start price"]
df_reports["difference_tp_start_price_percentage"] = (df_reports["adjusted_target_price"] - df_reports["start price"]).abs() / df_reports["start price"] * 100

In [1166]:
df_reports.to_csv("reports_data.csv")

In [1167]:
df_reports.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 916 entries, 0 to 915
Data columns (total 31 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   document_id                           916 non-null    int64  
 1   filename                              916 non-null    object 
 2   date                                  916 non-null    object 
 3   provider                              916 non-null    object 
 4   ticker                                916 non-null    object 
 5   company_name                          916 non-null    object 
 6   industry                              916 non-null    object 
 7   target_price                          705 non-null    float64
 8   rating                                916 non-null    object 
 9   start price                           916 non-null    float64
 10  one day after                         916 non-null    float64
 11  max price after 3 m

In [1168]:
df_grouped_rating = df_reports[df_reports['adjusted_target_price'] > 0]

df_grouped_rating = df_grouped_rating.groupby('rating', as_index=False).agg(
    total_reports=("filename", 'count'),
    average_difference=("difference_tp_start_price_percentage", 'mean'),
)


In [1169]:
df_grouped_rating

,rating,total_reports,average_difference
0,buy,228,34.581394
1,equal weight,73,15.322290
2,hold,57,13.772914
3,neutral,34,14.707363
4,none,1,10.344828
5,outperform,3,19.431983
6,overweight,276,22.621519
7,sell,7,15.418232
8,underperform,1,51.876805
9,underweight,25,29.105807
